In [1]:
from scipy.io import loadmat
lfp_buz_features = loadmat('lfp_buz_features_rgs14.mat')

## EEG Bands and EMG Like Signals

In [2]:
import pandas as pd
import numpy as np

lfp_matrix = np.array(lfp_buz_features['lfpFeatures'])

numColumns = len(lfp_matrix[0])

delta_pfc_df = pd.DataFrame(lfp_matrix[:, 0], columns=["DeltaPFC"])
theta_hpc_df = pd.DataFrame(lfp_matrix[:, 1], columns=["ThetaHPC"])
beta_pfc_df = pd.DataFrame(lfp_matrix[:, 2], columns=["BetaPFC"])
gamma_hpc_df = pd.DataFrame(lfp_matrix[:, 3], columns=["GammaHPC"])
emg_like_df = pd.DataFrame(lfp_matrix[:, 4], columns=["EMGLike"])

timesteps = 4216

In [3]:
delta_pfc_theta_hpc = delta_pfc_df['DeltaPFC'] / theta_hpc_df['ThetaHPC']
delta_pfc_beta_pfc  = delta_pfc_df['DeltaPFC'] / beta_pfc_df['BetaPFC']
delta_pfc_gamma_hpc = delta_pfc_df['DeltaPFC'] / gamma_hpc_df['GammaHPC']
theta_hpc_beta_pfc  = theta_hpc_df['ThetaHPC'] / beta_pfc_df['BetaPFC']
theta_hpc_gamma_hpc = theta_hpc_df['ThetaHPC'] / gamma_hpc_df['GammaHPC']
beta_pfc_gamma_hpc  = beta_pfc_df['BetaPFC'] / gamma_hpc_df['GammaHPC']

## Bands
- Nx7 data array, where N is the number of epochs, and columns refer to all possible ratios between the EEG bands & the EMG, with EMG being associated with the last column

In [4]:
rgs_14_bands = np.column_stack([
    delta_pfc_theta_hpc, 
    delta_pfc_beta_pfc, 
    delta_pfc_gamma_hpc, 
    theta_hpc_beta_pfc, 
    theta_hpc_gamma_hpc,
    beta_pfc_gamma_hpc,
    emg_like_df.EMGLike
])

In [5]:
rgs_14_bands.shape

(4216, 7)

In [6]:
rgs_14_bands.dtype

dtype('float64')

## EpochsLinked
- Nx4 data array, where N is the number of epochs, and columns are described as follows:

	- column 1: epoch ID
	- column 2: epoch index (currently not used)
	- column 3: ground truth sleep stage ID, where

				- 1 is associated with wakefulness,
				- 2 is associated with NREM sleep,
				- 3 is associated with REM sleep
	- column 4: the subject ID (used in multi-subject analysis only)

In [7]:
epoch_ids = np.arange(timesteps, dtype=int)

In [8]:
epoch_index = np.arange(timesteps, dtype=int)

In [9]:
ground_truth_sleep_stage_id = np.random.choice([1, 2, 3], size=timesteps).astype(int)

In [10]:
subject_id = np.ones(timesteps, dtype=int)

In [11]:
epochs_linked = np.column_stack([epoch_ids, epoch_index, ground_truth_sleep_stage_id, subject_id])

In [12]:
epochs_linked.shape

(4216, 4)

In [13]:
epochs_linked[0]

array([0, 0, 3, 1])

In [14]:
epochs_linked

array([[   0,    0,    3,    1],
       [   1,    1,    3,    1],
       [   2,    2,    1,    1],
       ...,
       [4213, 4213,    2,    1],
       [4214, 4214,    2,    1],
       [4215, 4215,    1,    1]])

## EpochTime
- Nx3 data array, where N is the number of epochs, and columns are described as follows:

	- column 1: epoch ID
	- column 2: recording mode (i.e. baseline or recovery), where
    
			   - 1 is associated with baseline,
			   - 2 is associated with recovery (after sleep deprivation)
	- column 3: the epoch date-time

In [15]:
recording_mode = np.random.choice([1, 2], size=timesteps).astype('<f8')

In [16]:
start_value = 41137.2918055555
step_size = 8.234392028162487e-05

time = np.arange(start_value, start_value + step_size * timesteps, step_size).astype('<f8')

In [17]:
time

array([41137.29180556, 41137.2918879 , 41137.29197024, ...,
       41137.63872048, 41137.63880283, 41137.63888517])

In [18]:
epoch_time = np.column_stack([epoch_ids.astype('<f8'), recording_mode, time])

In [19]:
epoch_time

array([[0.00000000e+00, 2.00000000e+00, 4.11372918e+04],
       [1.00000000e+00, 2.00000000e+00, 4.11372919e+04],
       [2.00000000e+00, 1.00000000e+00, 4.11372920e+04],
       ...,
       [4.21300000e+03, 2.00000000e+00, 4.11376387e+04],
       [4.21400000e+03, 2.00000000e+00, 4.11376388e+04],
       [4.21500000e+03, 2.00000000e+00, 4.11376389e+04]])

In [20]:
np.savez("rgs14.npz", d=rgs_14_bands, epochsLinked=epochs_linked, epochTime=epoch_time)